In [1]:
import numpy as np
from casadi import *
import do_mpc

In [2]:
model_type = 'discrete'
model = do_mpc.model.Model(model_type)

In [3]:
# Euler sampling interval
delta_t = 0.01
# Uncertain parameters: L is distance between wheels, r is the radius of the wheels
L = model.set_variable('_p', 'L')
r = model.set_variable('_p', 'r')

In [4]:
# input angular velocities of the left (u_l) and right (u_r) wheels
u_l = model.set_variable('_u', 'u_l')
u_r = model.set_variable('_u', 'u_r')

In [5]:
# linear (v) and angular (w) velocities of the center of the robot axle
v = (u_r + u_l) * r / 2
w = (u_r - u_l) * r / L

In [6]:
# state variables : position of the center of the robot axle and heading wrt x-axis
x = model.set_variable('_x', 'x')
y = model.set_variable('_x', 'y')
theta = model.set_variable('_x', 'theta')

In [7]:
# right-hand side of the dynamics
x_next = x + v * delta_t * cos(theta)
y_next = y + v * delta_t * sin(theta)
theta_next = theta + w * delta_t
model.set_rhs('x', x_next)
model.set_rhs('y', y_next)
model.set_rhs('theta', theta_next)

In [8]:
# auxiliary expression for the quadratic distance to the origin
model.set_expression('distance', x**2+y**2)
model.set_expression('zero',x-x)

SX(0)

In [9]:
model.setup()

In [10]:
mpc = do_mpc.controller.MPC(model)

In [16]:
setup_mpc = {
    'n_horizon': 10, # prediction horizion
    'n_robust': 1, # robust horizon
    'open_loop': 0, # if set to false, for each time step and scenario an individual control input it computed
    't_step': delta_t,  # timestep of the mpc
    'store_full_solution': True, # choose whether to store the full solution of the optimization problem
    # Use MA27 linear solver in ipopt for faster calculations:
    'nlpsol_opts': {'ipopt.linear_solver': 'mumps'}
}
mpc.set_param(**setup_mpc)

In [17]:
mterm = model.aux['distance'] # "naive" terminal cost
lterm = model.aux['zero']
mpc.set_objective(mterm=mterm,lterm=lterm) # "naive" cost function
mpc.set_rterm(u_l=0.1, u_r=0.1) # to be defined

In [18]:
# set the constraints of the control problem
mpc.bounds['lower','_u','u_l'] = -1
mpc.bounds['upper','_u','u_l'] = 1
mpc.bounds['lower','_u','u_r'] = -1
mpc.bounds['upper','_u','u_r'] = 1

In [19]:
L_values = np.array([0.48, 0.5, 0.53])
r_values = np.array([0.1, 0.15, 0.20])
mpc.set_uncertainty_values(L=L_values, r=r_values)

In [20]:
mpc.setup()

n_scenarios:  [1, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
Max:  9

-- Probability distribution modes -- 
1. Manual distrinution 
2. Hierarchical distribution 


Select prefered mode:  1



There are 9 scenarios, insert the aproximate probability value for each one by one.


Insert probability value for scenario 0:  3
Insert probability value for scenario 1:  2
Insert probability value for scenario 2:  4
Insert probability value for scenario 3:  2
Insert probability value for scenario 4:  6
Insert probability value for scenario 5:  2
Insert probability value for scenario 6:  1
Insert probability value for scenario 7:  2
Insert probability value for scenario 8:  6


[0.10714286 0.07142857 0.14285714 0.07142857 0.21428571 0.07142857
 0.03571429 0.07142857 0.21428571]
Final sum:  0.9999999999999999
